In [ ]:
!git clone https://github.com/Pakohp88/Equipo1_mlops.git

In [ ]:
%cd /content/Equipo1_mlops

In [ ]:
!git status

In [ ]:
!git checkout -b feature/eda

In [ ]:
import os
import re
import math
import json
import joblib
import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt
import seaborn as sns

from pathlib import Path
from typing import Tuple, List, Dict, Optional
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from typing import List, Dict, Optional

**Carga y revisión inicial del dataset**

In [ ]:
import pandas as pd

# Cargar el dataset (tu avance)
df = pd.read_csv('data/raw/turkish_music_emotion_modified.csv')

# Mostrar forma y columnas
print("Dimensiones del dataset:", df.shape)
print("\nColumnas del dataset:\n", list(df.columns))

# Tipos de datos y valores nulos (vista rápida)
print("\nInformación general del dataset:")
print(df.info())

# Primeras filas
print("\nVista previa:")
display(df.head())

# Conteo de valores faltantes
print("\nValores faltantes por columna:")
print(df.isnull().sum())

**Utilidades EDA**

In [ ]:
plt.rcParams["figure.figsize"] = (6,4)   # gráficos compactos
plt.rcParams["axes.grid"] = True

def normalizar_nombres_columnas(df: pd.DataFrame) -> pd.DataFrame:
    """Normaliza nombres (snake_case, sin acentos). No altera datos."""
    def norm(s: str) -> str:
        s = s.strip().lower()
        s = (s.replace("á","a").replace("é","e").replace("í","i")
               .replace("ó","o").replace("ú","u").replace("ñ","n"))
        s = re.sub(r'[^a-z0-9]+', '_', s)
        s = re.sub(r'_+','_', s).strip('_')
        return s
    return df.rename(columns={c: norm(c) for c in df.columns})

def resumen_dataset(df: pd.DataFrame) -> pd.DataFrame:
    """Resumen por columna: dtype, nulos, %, únicos, ejemplo."""
    total = len(df)
    resumen = []
    for c in df.columns:
        nulos = int(df[c].isna().sum())
        unicos = int(df[c].nunique(dropna=True))
        ejemplo = df[c].dropna().iloc[0] if df[c].dropna().shape[0] else None
        resumen.append({
            "columna": c,
            "tipo": str(df[c].dtype),
            "nulos": nulos,
            "%_nulos": round(nulos/total*100, 2),
            "unicos": unicos,
            "ejemplo": ejemplo
        })
    return pd.DataFrame(resumen).sort_values("%_nulos", ascending=False)

def detectar_columnas_id(df: pd.DataFrame) -> List[str]:
    """Heurística de columnas ID (para no analizarlas como variables)."""
    ids = []
    for c in df.columns:
        if re.search(r'(id|uuid|guid|folio|pk|code)$', c, flags=re.I):
            ids.append(c)
        if pd.api.types.is_integer_dtype(df[c]) and df[c].nunique(dropna=True) > 0.95*len(df):
            ids.append(c)
    return sorted(list(set(ids)))

def detectar_posible_objetivo(df: pd.DataFrame) -> Optional[str]:
    """Intenta adivinar columna objetivo por nombres comunes."""
    posibles = ["target", "label", "clase", "class", "emotion", "objetivo", "y"]
    candidatos = [c for c in df.columns if c.lower() in posibles]
    return candidatos[0] if candidatos else None

def estadisticas_descriptivas(df: pd.DataFrame, excluir: List[str]=[]) -> pd.DataFrame:
    """Describe numéricas (media, std, p25/p50/p75…) y cardinalidad de categóricas."""
    num_cols = [c for c in df.select_dtypes(include=np.number).columns if c not in excluir]
    cat_cols = [c for c in df.select_dtypes(exclude=np.number).columns if c not in excluir]
    desc_num = df[num_cols].describe().T if num_cols else pd.DataFrame()
    if not desc_num.empty:
        desc_num["nulos"] = df[num_cols].isna().sum()
    desc_cat = pd.DataFrame({
        "cardinalidad": df[cat_cols].nunique(dropna=True),
        "nulos": df[cat_cols].isna().sum()
    }) if cat_cols else pd.DataFrame()
    return desc_num, desc_cat

def distribuciones_numericas(df: pd.DataFrame, columnas: List[str], max_columnas: int=12, bins:int=20):
    """Histogramas para un subconjunto de columnas numéricas."""
    cols = columnas[:max_columnas]
    n = len(cols)
    if n == 0:
        print("No hay columnas numéricas para graficar.")
        return
    # cuadrícula razonable
    filas = int(np.ceil(n/3))
    fig, axes = plt.subplots(filas, 3, figsize=(15, 4*filas))
    axes = axes.flatten() if n>1 else [axes]
    for i, c in enumerate(cols):
        axes[i].hist(df[c].dropna(), bins=bins)
        axes[i].set_title(f"Distribución: {c}")
    for j in range(i+1, len(axes)):
        fig.delaxes(axes[j])
    plt.tight_layout()
    plt.show()

def distribuciones_categoricas(df: pd.DataFrame, columnas: List[str], top:int=10, max_columnas:int=12):
    """Barras de frecuencia para categóricas (top-k por columna)."""
    cols = columnas[:max_columnas]
    n = len(cols)
    if n == 0:
        print("No hay columnas categóricas para graficar.")
        return
    filas = int(np.ceil(n/3))
    fig, axes = plt.subplots(filas, 3, figsize=(15, 4*filas))
    axes = axes.flatten() if n>1 else [axes]
    for i, c in enumerate(cols):
        vc = df[c].value_counts(dropna=False).head(top)
        axes[i].bar(vc.index.astype(str), vc.values)
        axes[i].set_title(f"Frecuencias (top {top}): {c}")
        axes[i].tick_params(axis='x', rotation=45)
    for j in range(i+1, len(axes)):
        fig.delaxes(axes[j])
    plt.tight_layout()
    plt.show()

def matriz_correlacion(df: pd.DataFrame, excluir: List[str]=[]):
    """Mapa de calor de correlaciones para numéricas."""
    num_cols = [c for c in df.select_dtypes(include=np.number).columns if c not in excluir]
    if len(num_cols) < 2:
        print("Se necesitan ≥2 columnas numéricas para correlación.")
        return
    corr = df[num_cols].corr(numeric_only=True)
    plt.figure(figsize=(8,6))
    sns.heatmap(corr, cmap="coolwarm", center=0, linewidths=.5)
    plt.title("Matriz de correlación (numéricas)")
    plt.tight_layout()
    plt.show()

def boxplots_outliers_previos(df: pd.DataFrame, columnas: List[str], max_columnas:int=9):
    """Boxplots para visualizar outliers (previo a limpieza)."""
    cols = columnas[:max_columnas]
    n = len(cols)
    if n == 0:
        print("No hay columnas numéricas.")
        return
    filas = int(np.ceil(n/3))
    fig, axes = plt.subplots(filas, 3, figsize=(15, 4*filas))
    axes = axes.flatten() if n>1 else [axes]
    for i, c in enumerate(cols):
        axes[i].boxplot(df[c].dropna(), vert=True)
        axes[i].set_title(f"Boxplot: {c}")
    for j in range(i+1, len(axes)):
        fig.delaxes(axes[j])
    plt.tight_layout()
    plt.show()

def analisis_objetivo(df: pd.DataFrame, nombre_objetivo: Optional[str]) -> None:
    """Resumen del objetivo: balance de clases o stats numéricas."""
    if not nombre_objetivo or nombre_objetivo not in df.columns:
        print("No se identificó columna objetivo.")
        return
    y = df[nombre_objetivo]
    print(f"Columna objetivo detectada: {nombre_objetivo}")
    if y.dtype.kind in "ifu":   # numérica
        display(y.describe())
        plt.hist(y.dropna(), bins=20)
        plt.title(f"Distribución del objetivo: {nombre_objetivo}")
        plt.show()
    else:                        # categórica
        vc = y.value_counts(dropna=False)
        print("\nBalance de clases:")
        display(vc.to_frame("conteo").assign(porcentaje=lambda t: (t["conteo"]/len(y)*100).round(2)))
        plt.bar(vc.index.astype(str), vc.values)
        plt.title(f"Balance de clases: {nombre_objetivo}")
        plt.xticks(rotation=45)
        plt.show()

**EDA**

2.1 Normalizar solo los nombres

In [ ]:
# Normalizamos SOLO nombres de columnas para facilitar análisis (sin cambiar datos)
df_eda = normalizar_nombres_columnas(df.copy())
display(df_eda.head(3))

2.2 Resumen general por columna

In [ ]:
resumen = resumen_dataset(df_eda)
print("== Resumen por columna (ordenado por % de nulos) ==")
display(resumen.head(20))

Identificar columnas ID y objetivo

In [ ]:
cols_id = detectar_columnas_id(df_eda)
print("Columnas con posibles de ID:", cols_id)

objetivo = detectar_posible_objetivo(df_eda)
print("Posible columna objetivo:", objetivo)

Estadísticas descriptivas

In [ ]:
desc_num, desc_cat = estadisticas_descriptivas(df_eda, excluir=cols_id)
print("== Numéricas ==")
display(desc_num.head(20))
print("\n== Categóricas (cardinalidad & nulos) ==")
display(desc_cat.head(20))

Manejo de Nulos

In [ ]:
tabla_nulos = (
    df_eda.isna().sum().to_frame("nulos")
      .assign(porcentaje=lambda t: (t["nulos"]/len(df_eda)*100).round(2))
      .sort_values("nulos", ascending=False)
)
print("== Nulos por columna ==")
display(tabla_nulos.head(30))

Distribuciones: numéricas

In [ ]:
cols_num = [c for c in df_eda.select_dtypes(include=np.number).columns if c not in cols_id]
distribuciones_numericas(df_eda, columnas=cols_num, max_columnas=9, bins=20)

Distribuciones: categóricas

In [ ]:
cols_cat = [c for c in df_eda.select_dtypes(exclude=np.number).columns if c not in cols_id]
distribuciones_categoricas(df_eda, columnas=cols_cat, top=10, max_columnas=9)

Correlación

In [ ]:
matriz_correlacion(df_eda, excluir=cols_id)

Outliers (solo visualización previa, sin limpiar)

In [ ]:
boxplots_outliers_previos(df_eda, columnas=cols_num, max_columnas=9)

Análisis del objetivo

In [ ]:
analisis_objetivo(df_eda, objetivo)

**Limpieza y Preparación**

Configuración y rutas

In [ ]:
# Rutas de trabajo
RUTA_RAW   = Path("data/raw/turkish_music_emotion_modified.csv")
DIR_CLEAN  = Path("data/clean");  DIR_CLEAN.mkdir(parents=True, exist_ok=True)
DIR_PROC   = Path("data/processed"); DIR_PROC.mkdir(parents=True, exist_ok=True)
DIR_META   = Path("artifacts/meta"); DIR_META.mkdir(parents=True, exist_ok=True)

# Cargar el dataset crudo nuevamente (para partir de lo mismo de la EDA)
df = pd.read_csv(RUTA_RAW)
print("Dimensiones crudo:", df.shape)


Copia “no modificada”

In [ ]:
# 2.B GUARDAR COPIA "NO MODIFICADA" PARA COMPARACIÓN
RUTA_ORIGINAL = DIR_CLEAN / "dataset_original_sin_modificar.csv"
df.to_csv(RUTA_ORIGINAL, index=False, encoding="utf-8")
print("Copia no modificada guardada en:", RUTA_ORIGINAL)


Normalización de nombres y Limpieza de strings

In [ ]:
# 2.C NORMALIZAR NOMBRES Y LIMPIAR STRINGS (espacios / 'nan' / vacíos)
def normalizar_nombres_columnas(df: pd.DataFrame) -> pd.DataFrame:
    def norm(s: str) -> str:
        s = s.strip().lower()
        s = (s.replace("á","a").replace("é","e").replace("í","i")
               .replace("ó","o").replace("ú","u").replace("ñ","n"))
        s = re.sub(r'[^a-z0-9]+','_', s)
        s = re.sub(r'_+','_', s).strip('_')
        return s
    return df.rename(columns={c: norm(c) for c in df.columns})

def limpiar_textos(df: pd.DataFrame) -> pd.DataFrame:
    obj_cols = [c for c in df.columns if df[c].dtype == "object"]
    for c in obj_cols:
        df[c] = df[c].astype(str).str.strip()
        df[c] = df[c].replace({"": np.nan, "nan": np.nan, "None": np.nan})
    return df

df = normalizar_nombres_columnas(df.copy())
df = limpiar_textos(df)
print("Nombres normalizados y textos limpiados")


Eliminar filas inválidas del objetivo y duplicados

In [ ]:
# 2.E ELIMINAR REGISTROS INVÁLIDOS DE OBJETIVO Y DUPLICADOS
col_objetivo = "class"
filas_antes = len(df)
if col_objetivo in df.columns:
    df = df[~df[col_objetivo].isna()].copy()
    print(f"✔ Filas sin objetivo eliminadas: {filas_antes - len(df)}")

dup = int(df.duplicated().sum())
df = df.drop_duplicates().copy()
print("✔ Filas duplicadas eliminadas:", dup)


Reporte de nulos

In [ ]:
# 2.F REPORTE DE NULOS (ANTES DE IMPUTACIÓN)

tabla_nulos_antes = (
    df.isna().sum().to_frame("nulos")
      .assign(porcentaje=lambda t: (t["nulos"]/len(df)*100).round(2))
      .sort_values("nulos", ascending=False)
)
tabla_nulos_antes.to_csv(DIR_META / "nulos_antes.csv", encoding="utf-8")
print("Top nulos (antes):")
display(tabla_nulos_antes.head(15))


Imputación (mediana para numéricas, moda para categóricas)

In [ ]:
# 2.G IMPUTACIÓN DE NULOS (mediana / moda)
num_cols = [c for c in df.select_dtypes(include=np.number).columns]
cat_cols = [c for c in df.select_dtypes(exclude=np.number).columns]

# Medianas para numéricas
medianas = df[num_cols].median(numeric_only=True)
df[num_cols] = df[num_cols].fillna(medianas)

# Modas para categóricas
modas = {}
for c in cat_cols:
    moda = df[c].mode(dropna=True)
    modas[c] = (moda.iloc[0] if not moda.empty else "desconocido")
    df[c] = df[c].fillna(modas[c])

# Guardar diccionarios de imputación (para reproducibilidad)
(DIR_META / "imputacion_medianas.json").write_text(medianas.to_json(), encoding="utf-8")
(DIR_META / "imputacion_modas.json").write_text(json.dumps(modas, ensure_ascii=False, indent=2), encoding="utf-8")

print("Imputación realizada (medianas y modas)")


Manejo de Outliers

In [ ]:
# 2.H OUTLIERS: WINSORIZACIÓN POR IQR (REPORTE JSON)
def limites_iqr(s: pd.Series, k: float = 1.5):
    q1, q3 = s.quantile(0.25), s.quantile(0.75)
    iqr = q3 - q1
    return (q1 - k*iqr, q3 + k*iqr)

def winsorizar_iqr(df: pd.DataFrame, columnas: list, k: float = 1.5, guardar=True):
    reporte = {}
    for c in columnas:
        serie = df[c]
        if serie.notna().sum() == 0:
            continue
        lo, hi = limites_iqr(serie, k)
        antes = int(((serie < lo) | (serie > hi)).sum())
        df[c] = serie.clip(lower=lo, upper=hi)
        despues = int(((df[c] < lo) | (df[c] > hi)).sum())
        reporte[c] = {"antes": antes, "despues": despues, "lim_inf": float(lo), "lim_sup": float(hi)}
    if guardar:
        (DIR_META / "outliers_iqr.json").write_text(json.dumps(reporte, indent=2, ensure_ascii=False), encoding="utf-8")
    return df

df = winsorizar_iqr(df, num_cols, k=1.5, guardar=True)
print("Outliers tratados por IQR (reporte en artifacts/meta/outliers_iqr.json)")


Validaciones finales + guardar versión “limpia”

In [ ]:

# Comprobar nulos después
tabla_nulos_despues = (
    df.isna().sum().to_frame("nulos")
      .assign(porcentaje=lambda t: (t["nulos"]/len(df)*100).round(2))
      .sort_values("nulos", ascending=False)
)
tabla_nulos_despues.to_csv(DIR_META / "nulos_despues.csv", encoding="utf-8")

print("Nulos (después) - top:")
display(tabla_nulos_despues.head(10))

# Guardar CSV/parquet limpios
RUTA_LIMPIO_CSV = DIR_CLEAN / "dataset_limpio.csv"
RUTA_LIMPIO_PAR = DIR_CLEAN / "dataset_limpio.parquet"
df.to_csv(RUTA_LIMPIO_CSV, index=False, encoding="utf-8")
df.to_parquet(RUTA_LIMPIO_PAR, index=False)

print("✅ Dataset limpio guardado en:")
print("  -", RUTA_LIMPIO_CSV)
print("  -", RUTA_LIMPIO_PAR)


Separación X/y y exportar para modelado

In [ ]:

col_objetivo = "class"  # ajusta si corresponde
if col_objetivo in df.columns:
    y = df[col_objetivo]
    X = df.drop(columns=[col_objetivo])

    # Guardar para el punto de modelado
    (DIR_PROC / "X.csv").write_text(X.to_csv(index=False), encoding="utf-8")
    (DIR_PROC / "y.csv").write_text(y.to_csv(index=False, header=True), encoding="utf-8")
    print("✔ Archivos para modelado guardados en data/processed/: X.csv y y.csv")
else:
    print("ℹ️ No se encontró columna objetivo 'class'; omito exportación X/y.")


Versionado con DVC

In [ ]:

# 2.K.1 ASEGURAR QUE DVC ESTÉ INSTALADO

import importlib, sys, subprocess

def asegurar_paquete(paquete: str):
    try:
        importlib.import_module(paquete)
        print(f" {paquete} ya está instalado.")
    except ImportError:
        print(f"Instalando {paquete} ...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", paquete])
        importlib.invalidate_caches()
        importlib.import_module(paquete)
        print(f"{paquete} instalado.")

asegurar_paquete("dvc")


Versionado

In [ ]:
%%bash
set -e

echo "== 2.K.2 Versionado con DVC (libreta) =="

# 1) Inicializar DVC SOLO si no existe
if [ ! -d ".dvc" ]; then
  echo "Inicializando DVC..."
  dvc init -q
else
  echo "DVC ya estaba inicializado."
fi

# 2) Asegurar que existen los archivos esperados antes de agregarlos
ls -lah data/clean || true
ls -lah artifacts/meta || true

# 3) Agregar con DVC los artefactos clave de esta fase
#    (ajusta o comenta los que no existan en tu caso)
echo "Agregando datasets y reportes a DVC..."
dvc add data/raw/turkish_music_emotion_modified.csv || true
dvc add data/clean/dataset_original_sin_modificar.csv || true
dvc add data/clean/dataset_limpio.csv || true
dvc add data/clean/dataset_limpio.parquet || true
dvc add artifacts/meta/nulos_antes.csv || true
dvc add artifacts/meta/nulos_despues.csv || true
dvc add artifacts/meta/outliers_iqr.json || true
dvc add artifacts/meta/imputacion_medianas.json || true
dvc add artifacts/meta/imputacion_modas.json || true

# (Opcional) si en 2.J guardaste X/y:
if [ -f "data/processed/X.csv" ]; then dvc add data/processed/X.csv; fi
if [ -f "data/processed/y.csv" ]; then dvc add data/processed/y.csv; fi

# 4) Si estamos en un repositorio Git, versionar .dvc y .gitignore
if git rev-parse --is-inside-work-tree >/dev/null 2>&1; then
  echo "Repositorio Git detectado: agregando y haciendo commit…"
  git add .gitignore *.dvc */*.dvc */*/*.dvc || true
  git add data/clean/*.csv.dvc data/clean/*.parquet.dvc artifacts/meta/*.dvc || true
  git add data/raw/*.csv.dvc data/processed/*.dvc || true

  git commit -m "DVC: datasets con EDA y Limpio + reportes de nulos y outliers"
else
  echo "No se detectó repositorio Git. Se omitió 'git add/commit'."
  echo "   Sugerencia: corre 'git init' y vuelve a ejecutar esta celda para trackear las .dvc con Git."
fi

echo "Listo: archivos registrados con DVC."
echo "   (Si ya tienes remoto DVC configurado, puedes 'dvc push' en otra celda.)"


**Configuracion DVC remoto**

Instalar el plugin de Azure

In [ ]:
import sys, subprocess
subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "--upgrade", "dvc-azure"])
print("dvc-azure listo")


Configuracion de SAS de Azure

In [ ]:
%%bash
set -e

# Datos de tu Azure Blob
export AZURE_STORAGE_ACCOUNT="mlopsdvc"
export AZURE_STORAGE_CONTAINER="dvcdata"
# Pega tu SAS tal cual (incluye el '?' inicial)
export AZURE_SAS_TOKEN="?sp=rwdl&st=2025-10-08T20:17:13Z&se=2026-01-02T04:32:13Z&spr=https&sv=2024-11-04&sr=c&sig=N4S%2FLFOyQ5q%2BsrHkgx3zKhZ3%2BiktEYy5smHLGZE3pvY%3D"

# Inicializa DVC si aún no existe la carpeta .dvc (no toca git init)
[ -d ".dvc" ] || dvc init -q

# Define/actualiza el remoto por defecto 'azdatos' apuntando al contenedor (sin subcarpeta)
REMOTE_URL="azure://${AZURE_STORAGE_CONTAINER}"
if ! dvc remote list | grep -q "^azdatos"; then
  dvc remote add -d azdatos "$REMOTE_URL"
else
  dvc remote modify azdatos url "$REMOTE_URL"
fi

# Config pública (no secreta) y SAS en config.local (no se sube a git)
dvc remote modify azdatos account_name "$AZURE_STORAGE_ACCOUNT"
dvc remote modify --local azdatos sas_token "$AZURE_SAS_TOKEN"

# Push directo al remoto
dvc push

echo "dvc push completado en azure://${AZURE_STORAGE_CONTAINER} (cuenta: $AZURE_STORAGE_ACCOUNT)"


Push a git

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
%%bash
set -e
echo "🔎 Buscando Fase1_AvanceProyecto.ipynb en rutas comunes (incluye Drive montado)..."
for p in /content "/content/Colab Notebooks" "/content/drive/MyDrive" "/content/drive/MyDrive/Colab Notebooks"; do
  if [ -d "$p" ]; then
    echo; echo "📂 $p"
    find "$p" -maxdepth 6 -type f -name "Fase1_AvanceProyecto.ipynb" 2>/dev/null || true
  fi
done
